# Libraries

In [1]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.constants as cnts
import time
import os

In [2]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Constants

In [24]:
hbar = cnts.hbar # Js
a0 = cnts.physical_constants['Bohr radius'][0] # m
m6Li2 = 2*9.98834e-27 # kg
# 650G -> 810.9
# 690G -> 1423.1
# 710G -> 1887.4
# 750G -> 3515.7
# 800G -> 11336.4
# 815G -> 22594.2

as_6Li2 = 0.6*1423.1*a0 # m

N = 35e3 # atoms

omega_r = 2*np.pi*163.0 #  Hz
omega_z = 2*np.pi*11.0  # Hz

Omega1 = 2*omega_r # Hz
Omega2 = Omega1*np.pi/2 # Hz

h = 0.5e-6 # m
dr = h # m
dz = h # m

r_max = 64*dr # m
z_max = 512*dz # m
print(1e6*r_max, 1e6*z_max)
print(1e6*dr, 1e6*dz)

32.0 256.0
0.5 0.5


In [25]:
kFermi = np.sqrt(2*m6Li2*(omega_r**2*omega_z)**(1.0/3.0)/hbar)*(6*N)**(1.0/6.0)
inv_kFa = 1./(kFermi*(as_6Li2/0.6))
inv_kFa

4.333482785633333

# Dimensionless constants

In [26]:
# ----- Characteristics ------
omega = Omega1
x_s = np.sqrt(hbar/(m6Li2*omega))

# ----- Distances ------
r_adim_max  = r_max/x_s
z_adim_max  = z_max/x_s
dr_adim = dr/x_s
dz_adim = dz/x_s

# ----- Frequencies ------
beta_r = omega_r/omega
beta_z = omega_z/omega
sigma1 = Omega1/omega
sigma2 = Omega2/omega

# ----- System ------
eta = 1.0
C_eta = 4.0*np.pi*N*as_6Li2/x_s
#eta = 2.0/3.0
#C_eta = 0.37*0.5*(3*np.pi**2*N)**(2./3)

# Simulation
## Arrays

In [27]:
r = cp.arange(-r_adim_max, r_adim_max, dr_adim)
z = cp.arange(-z_adim_max, z_adim_max, dz_adim)
X, Y, Z = cp.meshgrid(r, r, z, indexing='ij')
RR = X**2+Y**2
X, Y = [], []
print(RR.shape)

kr = 2.0*np.pi*cp.fft.fftfreq(len(r),dr_adim)
kz = 2.0*np.pi*cp.fft.fftfreq(len(z),dz_adim)
Kx, Ky, Kz = cp.meshgrid(kr, kr, kz, indexing='ij')
KK = Kx**2+Ky**2+Kz**2
Kx, Ky, Kz = [], [], []

r_center_idx = np.where(np.abs(r)<0.5*dr_adim)[0][0]
z_center_idx = np.where(np.abs(z)<0.5*dz_adim)[0][0]
print(r[r_center_idx], z[z_center_idx])

(128, 128, 1024)
0.0 0.0


In [28]:
def Gauss_psi0(RR, Z):
    global omega_r,omega_z, N, as_6Li2, x_s, m6Li2
    omega_bar = (omega_r*omega_r*omega_z)**(1./3.)
    mu  = 0.5*hbar*omega_bar*(15*N*as_6Li2/x_s)**(2./5.)
    rTF_r, rTF_z = cp.sqrt(2.0*mu/(m6Li2*omega_r**2)), cp.sqrt(2.0*mu/(m6Li2*omega_z**2))
    return mu, cp.exp(-2*RR*(x_s/rTF_r)**2-2*(x_s*Z/rTF_z)**2)

def normalization(psi, dr, dz):
    return cp.sum(cp.conj(psi)*psi).real*dr*dr*dz

def psi_error(psi_analytic, psi_numeric, dr, dz):
    return normalization(psi_analytic-psi_numeric, dr, dz)

def TSSP_step_3D_imag_time_fftn(psi0,RR,Z,KK,t,dt,int_potential_dt,pot_args,dr,dz,C_eta,eta,gamma):
    U_int = int_potential_dt(RR, Z, t, dt, pot_args)
    psi = psi0
    psi = cp.exp(0.5*(-1.0)*(U_int+dt*C_eta*(psi*cp.conj(psi))**eta))*psi
    psi = cp.fft.ifftn(cp.exp((-1.0)*dt*(KK)/2.0)*cp.fft.fftn(psi, axes=(0,1,2,)), axes=(0,1,2,))
    psi = cp.exp(0.5*(-1.0)*(U_int+dt*C_eta*(psi*cp.conj(psi))**eta))*psi
    return psi/cp.sqrt(normalization(psi, dr, dz))

def TSSP_step_3D_real_time_fftn(psi0,RR,Z,KK,t,dt,int_potential_dt,pot_args,dr,dz,C_eta,eta,gamma):
    U_int = int_potential_dt(RR, Z, t, dt, pot_args)
    f_gamma = (1.0j+gamma)/(1.0+gamma**2)
    psi = psi0
    psi = cp.exp(0.5*(-f_gamma)*(U_int+dt*C_eta*(psi*cp.conj(psi))**eta))*psi
    psi = cp.fft.ifftn(cp.exp((-f_gamma)*dt*(KK)/2.0)*cp.fft.fftn(psi, axes=(0,1,2,)), axes=(0,1,2,))
    psi = cp.exp(0.5*(-f_gamma)*(U_int+dt*C_eta*(psi*cp.conj(psi))**eta))*psi
    return psi/cp.sqrt(normalization(psi, dr, dz))

def harmonic_potential_biExcitation_time_integral(RR, Z, t, dt, pot_args):
    b_r, b_z, a1, s1, a2, s2 = pot_args
    U0_T = 0.5*dt*(RR*b_r**2 + (b_z*Z)**2)
    Umex1_T = a1*(cp.cos(s1*(t+dt))-cp.cos(s1*t))/s1
    Umex2_T = a2*(cp.cos(s2*(t+dt))-cp.cos(s2*t))/s2
    return U0_T + 0.5*(RR*b_r**2)*(Umex1_T+Umex2_T)

def harmonic_potential_barrier_biExcitation_time_integral(RR, Z, t, dt, pot_args):
    b_r, b_z, a1, s1, a2, s2, VBarrier, Size = pot_args
    U0_T = 0.5*dt*(RR*b_r**2 + (b_z*Z)**2)
    U0_Barrier = 0.5*dt*VBarrier*cp.exp(-(Z/Size)**2)
    Umex1_T = a1*(cp.cos(s1*(t+dt))-cp.cos(s1*t))/s1
    Umex2_T = a2*(cp.cos(s2*(t+dt))-cp.cos(s2*t))/s2
    return U0_T + U0_Barrier + 0.5*(RR*b_r**2)*(Umex1_T+Umex2_T)

In [ ]:
start_time = time.time()
dt_adim = 0.01
mu_adim, ground_state = Gauss_psi0(RR, Z)
ground_state = ground_state/cp.sqrt(normalization(ground_state, dr_adim, dz_adim))
eps = 1.0

B_size = 10e-6/x_s
B_height = 2*mu_adim/(m6Li2*omega**2*x_s**2)

count = 0
while eps > 1e-8:
    count += 1
    psi_step = TSSP_step_3D_imag_time_fftn(psi0=ground_state, RR=RR, Z=Z, KK=KK, t=0.0, dt=dt_adim, 
                                           int_potential_dt=harmonic_potential_biExcitation_time_integral, 
                                           pot_args=cp.array([beta_r,beta_z,0.0,1.0,0.0,1.0]), 
                                           dr=dr_adim, dz=dz_adim, C_eta=C_eta, eta=eta, gamma=0)
    
    eps = cp.abs(psi_error(psi_step, ground_state, dr_adim, dz_adim))
    ground_state = psi_step

print(count, eps)
psi_step = []
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12,4))
im0=ax[0].imshow(cp.asnumpy(ground_state[:,r_center_idx,:]).real, aspect='auto', origin='lower',
             extent=[1e6*x_s*z[0],1e6*x_s*z[-1],1e6*x_s*r[0],1e6*x_s*r[-1]])
ax[0].set_xlabel(r"z [$\mu$m]",fontsize=15)
ax[0].set_ylabel(r"x [$\mu$m]",fontsize=15)
plt.colorbar(im0, ax=ax[0])

im1=ax[1].imshow(cp.asnumpy(ground_state[:,:,z_center_idx]).real, aspect='auto', origin='lower',
             extent=[1e6*x_s*r[0],1e6*x_s*r[-1],1e6*x_s*r[0],1e6*x_s*r[-1]])
ax[1].set_xlabel(r"y [$\mu$m]",fontsize=15)
ax[1].set_ylabel(r"x [$\mu$m]",fontsize=15)
plt.colorbar(im1, ax=ax[1])

plt.tight_layout()

In [ ]:
dt_adim = 2*np.pi*0.001
dt_save = 2*np.pi*0.05
t_adim_max = 2*np.pi*50.0
t_adim = np.arange(0,t_adim_max,dt_adim)
t_to_save = t_adim[np.mod(range(len(t_adim)), int(dt_save/dt_adim))==0]
print(round(1e3*t_adim_max/(omega),3), round(1e3*dt_adim/(omega),3), len(t_adim), len(t_to_save))

153.3742331288344 50000 1000


In [ ]:
gamma = 0.005
alpha1 = 0.10
alpha2 = alpha1*0

filename = "/content/drive/MyDrive/Colab Notebooks/Data/data_UFG_832G_10p_g0p005"
sim_file = filename+"_psi_vs_time"
column_density_file = filename+"_column_density"
params_file = filename+"_parameters"

parameters = np.array([omega, x_s, r_adim_max, z_adim_max, dr_adim, dz_adim, beta_r, beta_z, alpha1, alpha2, 
                       sigma1, sigma2, eta, C_eta, m6Li2, as_6Li2, N, gamma, t_adim, t_to_save], dtype=object)
np.save(params_file, parameters)

start_time = time.time()
psi_step_temp = ground_state
psi_steps = [psi_step_temp[r_center_idx,:,:]]
nc_steps = [dr_adim*cp.sum(cp.abs(psi_step_temp)**2,axis=0)]

t = 0.0
for ts in tqdm(range(1,len(t_to_save))):
    for i in range(int(dt_save/dt_adim)):
        t += dt_adim
        psi_step_temp = TSSP_step_3D_real_time_fftn(psi0=psi_step_temp, RR=RR, Z=Z, KK=KK, t=t, dt=dt_adim, 
                                                    int_potential_dt=harmonic_potential_biExcitation_time_integral, 
                                                    pot_args=cp.array([beta_r,beta_z,alpha1,sigma1,alpha2,sigma2]), 
                                                    dr=dr_adim, dz=dz_adim, C_eta=C_eta, eta=eta, gamma=gamma)
    psi_steps.append(psi_step_temp[r_center_idx,:,:])
    nc_steps.append(dr_adim*cp.sum(cp.abs(psi_step_temp)**2,axis=0))

    if cp.mod(ts+1,len(t_to_save)/10)==0:
        sim_file = filename+f"_{ts+1}"+"_psi_vs_time"
        column_density_file = filename+f"_{ts+1}"+"_column_density"
        print(ts+1)
        #save psi
        cp.save(sim_file, cp.array(psi_steps))
        #save column density
        cp.save(column_density_file, cp.array(nc_steps))
        psi_steps, nc_steps = [], []
        
print("--- %s seconds ---" % (time.time() - start_time))

100
200
300
400
500
600
700
800
900
1000

--- 9924.329272031784 seconds ---
